In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import os
import scipy as sp
from collections import defaultdict
from itertools import product
import gurobipy as gb
from gurobipy import GRB

In [2]:
'''k = 7
locs = pd.read_json('./cleandata/locations.json').loc[:124]
longs = locs['long'].to_numpy()
lats = locs['lat'].to_numpy()
long_lat_in_one = np.array([*zip(longs[1:],lats[1:])])

centroids, labels = sp.cluster.vq.kmeans2(long_lat_in_one, k=k)
clusters = defaultdict(list)
for node,label in enumerate(labels):
    clusters[label].append(node+1)
clusters = list(clusters.values())
cleaned_clusters = []
for cluster in clusters:
    if len(cluster) > 10:
        cleaned_clusters.extend(np.array_split(cluster, len(cluster)//6))
    else:
        cleaned_clusters.append(cluster)
        
print(len(clusters))
print(*map(lambda x: len(x), cleaned_clusters))
#Ignore the warning, if you see one
clusters = cleaned_clusters
print(cleaned_clusters)'''

7
6 6 11 7 6 6 6 6 6 6 6 6 6 6 6 9 8 8 3
[array([ 1, 21, 48, 49, 50, 55]), array([ 58,  59,  62,  63, 122, 124]), array([  2,   3,  12,  51,  60,  61,  66,  86,  88,  89, 123]), array([ 4,  5,  6,  9, 11, 14, 15]), array([18, 19, 22, 24, 26, 28]), array([31, 32, 33, 34, 35, 36]), array([37, 38, 40, 41, 42, 43]), array([67, 68, 69, 73, 75, 77]), array([78, 79, 81, 82, 84, 85]), array([87, 92, 93, 94, 95, 96]), array([103, 104, 105, 106, 107, 108]), array([109, 110, 111, 112, 113, 114]), array([115, 116, 117, 118, 119, 120]), array([ 7, 10, 16, 17, 44, 45]), array([ 46,  47,  56,  57, 101, 102]), array([ 8, 13, 20, 23, 25, 27, 71, 72, 74]), array([ 76,  80,  83,  90,  97,  98,  99, 100]), [29, 30, 39, 52, 53, 54, 91, 121], [64, 65, 70]]


In [19]:
'''cc = []
for cluster in cleaned_clusters:
    if isinstance(cluster, np.ndarray):
        cc.append(cluster.tolist())
    else:
        cc.append(cluster)
json_ob = json.dumps(cc, indent=4)
with open('./cluster_json_7.json', 'w') as file:
    file.write(json_ob)'''
with open('./cluster_json_7.json', 'r') as file:
    clusters = json.load(file)
clusters
print(*map(lambda x: len(x), clusters))

6 6 11 7 6 6 6 6 6 6 6 6 6 6 6 9 8 8 3


In [4]:
c = pd.read_json('./cleandata/dist_mat.json')
t = pd.read_json('./cleandata/time_mat.json')
locdem = pd.read_excel('./cleandata/locdem.xlsx')
#8 - 125,11 - 126
c = c.loc[:124, :124].to_numpy()
t = t.loc[:124, :124].to_numpy()
q = locdem['Number of pallets']
q = np.array([0]+list(q))

def get(i):
    if i == 125:
        return 8
    if i == 126:
        return 11
    return i

def time(i,j):
    i,j = get(i), get(j)
    return t[i,j]

def cost(i,j):
    if (i,j) in [(126,11), (11,126), (8,125), (125,8)]:
        return 1e+8
    i,j = get(i), get(j)
    return c[i,j]

Q = 9
m = gb.Model()
x = m.addVars(q.size,q.size,vtype=GRB.BINARY, name='x')
L = m.addVars(len(clusters), vtype=GRB.INTEGER, name='L')
u = m.addVars(q.size, lb=0.0, vtype=GRB.CONTINUOUS, name='u')
iter_count = 0

non_wh = range(1, q.size)
arcs = filter(lambda tup: tup[0] != tup[1], product(non_wh,non_wh))
for i,j in arcs:
    m.addConstr(u[i] - u[j] + Q*x[i,j] <= Q - q[j])
for i in range(q.size):
    m.addConstr(u[i] >= q[i])  

m.setObjective(sum([x[i,j] * cost(i,j) for i,j in product(range(q.size), range(q.size))]), GRB.MINIMIZE)

for cluster in clusters.values():
    N = list(sorted(cluster))
    if 8 in N:
        N.append(124)
    if 11 in N:
        N.append(125)
        
    V = [0] + N.copy()
    v = len(V)
    A = filter(lambda tup: tup[0] != tup[1], product(N,N))
    def Del(i):
        F = V.copy()
        F.remove(i)
        return F
    
    for i in N:
        m.addConstr(sum([x[i,j] for j in Del(i)]) == 1)
    for j in N:
        m.addConstr(sum([x[i,j] for i in Del(j)]) == 1)
    
    m.addConstr(sum([x[0,j] for j in Del(0)]) == L[iter_count])
    iter_count+=1
    

In [5]:
#m.optimize()
print(len(clusters))

19


In [6]:
from tqdm import tqdm

In [7]:
Q = 9
iter_count = 0
primary_models = []
Ls = []

for cluster in clusters:
    Neighbours = list(sorted(cluster))
    if 8 in Neighbours:
        Neighbours.append(124)
    if 11 in Neighbours:
        Neighbours.append(125)
        
    all_nodes = [0] + Neighbours.copy()
    v = len(all_nodes)
    V = list(range(v))
    N = V.copy()
    N.remove(0)
    
    def Del(i):
        F = V.copy()
        F.remove(i)
        return F
    
    m = gb.Model()
    x = m.addVars(v,v, vtype=GRB.BINARY, name='x')
    m.setObjective(sum([cost(all_nodes[i], all_nodes[j]) * x[i,j] for i,j in product(V,V)]))
    for i in N:
        m.addConstr(sum([x[i,j] for j in Del(i)]) == 1)
    for j in N:
        m.addConstr(sum([x[i,j] for i in Del(j)]) == 1)
        
    l = m.addVar(lb=0.0, vtype=GRB.INTEGER)
    Ls.append(l)
    m.addConstr(sum([x[0,j] for j in Del(0)]) == l)
    
    u = m.addVars(V, vtype=GRB.CONTINUOUS, name='u')
    for i,j in filter(lambda tup: tup[0] != tup[1], product(N,N)):
        m.addConstr(u[i] - u[j] + Q*x[i,j] <= Q - q[j])
    for i in V:
        m.addConstr(u[i] >= q[i])
    
    primary_models.append(m)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-12


In [8]:
print(len(primary_models))
for model in tqdm(primary_models):
    model.optimize()

19


  0%|                                                    | 0/19 [00:00<?, ?it/s]

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 50 rows, 57 columns and 176 nonzeros
Model fingerprint: 0xc84d5f74
Variable types: 7 continuous, 50 integer (49 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e+03, 3e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]
Found heuristic solution: objective 226239.00000
Presolve removed 8 rows and 21 columns
Presolve time: 0.00s
Presolved: 42 rows, 36 columns, 150 nonzeros
Variable types: 6 continuous, 30 integer (30 binary)
Found heuristic solution: objective 115451.00000

Root relaxation: objective 4.925459e+04, 11 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 49254.5926    0   12 115451.000 49254.5926  57.3

 11%|████▋                                       | 2/19 [00:00<00:01, 11.00it/s]

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 145 rows, 157 columns and 596 nonzeros
Model fingerprint: 0xf64903bf
Variable types: 12 continuous, 145 integer (144 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [7e+01, 1e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]
Found heuristic solution: objective 1362496.0000
Presolve removed 13 rows and 14 columns
Presolve time: 0.00s
Presolved: 132 rows, 143 columns, 572 nonzeros
Variable types: 11 continuous, 132 integer (132 binary)

Root relaxation: objective 1.593468e+05, 32 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 159346.778    0   19 1362496.00 159346.778  88.3%     -    0s
H    0     0            

...

 89%|██████████████████████████████████████▍    | 17/19 [03:26<00:19,  9.73s/it]

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 82 rows, 91 columns and 314 nonzeros
Model fingerprint: 0x6005c982
Variable types: 9 continuous, 82 integer (81 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [5e+02, 6e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]
Found heuristic solution: objective 792000.00000
Presolve removed 10 rows and 27 columns
Presolve time: 0.00s
Presolved: 72 rows, 64 columns, 280 nonzeros
Variable types: 8 continuous, 56 integer (56 binary)
Found heuristic solution: objective 316386.00000

Root relaxation: objective 1.377667e+04, 27 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 13776.6667    0   14 316386.000 13776.6667  95.

 95%|████████████████████████████████████████▋  | 18/19 [03:29<00:07,  7.59s/it]

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 17 rows, 21 columns and 44 nonzeros
Model fingerprint: 0xefe28341
Variable types: 4 continuous, 17 integer (16 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [4e+03, 6e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]
Found heuristic solution: objective 299226.00000
Presolve removed 5 rows and 12 columns
Presolve time: 0.00s
Presolved: 12 rows, 9 columns, 30 nonzeros
Variable types: 3 continuous, 6 integer (6 binary)
Found heuristic solution: objective 105018.00000

Root relaxation: objective 2.224930e+04, 6 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 22249.2963    0    6 105018.000 22249.2963  78.8%     

100%|███████████████████████████████████████████| 19/19 [03:29<00:00, 11.02s/it]


In [22]:
s = 0
for model in primary_models:
    s += model.objVal
print(s)

4226491.0


In [23]:
4.896489e+6 - s

669998.0

In [35]:
for l in Ls:
    print(l.x)

2.0
2.0
3.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
3.0
2.0
2.0
1.0


In [37]:
model_solutions = {}
for i,model in enumerate(primary_models):
    soln = json.loads(model.getJSONSolution())
    model_solutions[i] = soln

In [38]:
json.dump(model_solutions, open('./'))

{0: {'SolutionInfo': {'Status': 2,
   'Runtime': 0.08168792724609375,
   'Work': 0.009664164612575389,
   'ObjVal': 97405,
   'ObjBound': 97405,
   'ObjBoundC': 97405,
   'MIPGap': 0,
   'IntVio': 0,
   'BoundVio': 0,
   'ConstrVio': 3.552713678800501e-15,
   'IterCount': 906,
   'BarIterCount': 0,
   'NodeCount': 155,
   'SolCount': 6,
   'PoolObjBound': 97405,
   'PoolObjVal': [97405, 97418, 97519, 100101, 115451, 226239]},
  'Vars': [{'VarName': 'x[0,1]', 'X': 1},
   {'VarName': 'x[0,6]', 'X': 1},
   {'VarName': 'x[1,4]', 'X': 1},
   {'VarName': 'x[2,5]', 'X': 1},
   {'VarName': 'x[3,0]', 'X': 1},
   {'VarName': 'x[4,3]', 'X': 1},
   {'VarName': 'x[5,0]', 'X': 1},
   {'VarName': 'x[6,2]', 'X': 1},
   {'VarName': 'C49', 'X': 2},
   {'VarName': 'u[1]', 'X': 2},
   {'VarName': 'u[2]', 'X': 5},
   {'VarName': 'u[3]', 'X': 4},
   {'VarName': 'u[4]', 'X': 3},
   {'VarName': 'u[5]', 'X': 7.9999999999999964},
   {'VarName': 'u[6]', 'X': 3}]},
 1: {'SolutionInfo': {'Status': 2,
   'Runtime':